In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
'''
@File    :   Untitled-1
@Time    :   2023/07/27 12:49:14
@Author  :   Siddharth Ravi
@Version :   1.0
@Contact :   siddharth.ravi@ua.es
@License :   (C)Copyright 2022-2023, Siddharth Ravi, Distributed under terms of the MIT license
@Desc    :   Find and blur person in an image using a pretrained model with pytorch
'''

import torchvision
from torchvision.transforms.functional import to_pil_image
from torchvision.models.segmentation import DeepLabV3_ResNet101_Weights, deeplabv3_resnet101
from torchvision.io.image import read_image
import numpy as np
import cv2
import torch

In [3]:
def blur_person(img, mask):
    # Change image from tensor to numpy array
    img = img.numpy()
    mask = mask.numpy()
    print(img.shape, mask.shape)
    masked_img = cv2.bitwise_and(img, mask)
    cv2.imwrite("masked_img.jpg", masked_img)

In [4]:

def find_person(image, model):
    # Find the person in the image using deeplabv3_resnet101
    weights = DeepLabV3_ResNet101_Weights.DEFAULT
    model = deeplabv3_resnet101(weights=weights)
    model.eval()

    # Step 2: Initialize the inference transforms
    preprocess = weights.transforms()

    # Step 3: Apply inference preprocessing transforms
    batch = preprocess(image).unsqueeze(0)

    # Step 4: Use the model and visualize the prediction
    prediction = model(batch)["out"]
    normalized_masks = prediction.softmax(dim=1)
    class_to_idx = {cls: idx for (idx, cls) in enumerate(weights.meta["categories"])}

    mask = normalized_masks[0, class_to_idx["person"]]
    print(mask.shape, image.shape)
    # Threshold the mask
    mask = mask > 0.5
    mask = mask.numpy().astype(np.uint8)
    mask = mask * 255
    
    mask = torch.from_numpy(mask)
    return mask

In [5]:

model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
model.eval()
image = read_image("1663943484522.jpg")
mask = find_person(image, model)
blur_person(image, mask)



/Users/sid/mambaforge/envs/d2l/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sid/mambaforge/envs/d2l/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([520, 924]) torch.Size([3, 720, 1280])
(3, 720, 1280) (520, 924)


error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/core/src/arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'binary_op'
